In [1]:
import sys
import datetime

import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

sys.path.append("..")

from utils import paths

from sklearn.model_selection import train_test_split

pd.set_option("display.max_columns", None)

tf.config.list_physical_devices()


2023-07-05 23:15:04.853738: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 23:15:04.921315: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-05 23:15:04.939323: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-05 23:15:05.298860: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
data = pd.read_csv(paths.DATA_PATH + "/COVID19MX_sample.csv")
data.sample(10)


,SEXO,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,DEFUNCION,INTUBADO,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,CLASIFICACION_FINAL,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
46649,1,2,2022-01-19,2022-01-19,1,NaN,2.0,59.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,3,109.0,0.0,NaN
11510,2,2,2022-09-09,2022-09-08,1,2.0,1.0,24.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7,109.0,0.0,2.0
33847,2,1,2022-10-31,2022-10-29,0,3.0,2.0,42.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3,109.0,0.0,3.0
27186,1,2,2023-01-28,2023-01-27,1,2.0,NaN,75.0,2.0,2.0,1.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,NaN,6,109.0,0.0,2.0
15668,1,2,2022-06-05,2022-05-03,0,2.0,1.0,14.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7,109.0,0.0,2.0
22101,1,2,2022-01-31,2022-01-20,1,2.0,1.0,54.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,3,109.0,0.0,2.0
37710,2,2,2022-01-14,2022-01-01,1,2.0,1.0,51.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,7,109.0,0.0,2.0
50895,2,2,2022-01-28,2022-01-24,1,2.0,1.0,82.0,2.0,2.0,1.0,2.0,2.0,1.0,NaN,2.0,2.0,2.0,2.0,2.0,3,109.0,0.0,2.0
68866,1,2,2022-01-29,2022-01-27,1,2.0,2.0,47.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3,109.0,0.0,2.0
72349,2,2,2022-01-12,2022-01-09,1,2.0,1.0,69.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,3,109.0,0.0,2.0


In [3]:
X = data.drop(columns=["DEFUNCION", "FECHA_INGRESO", "FECHA_SINTOMAS"]).to_numpy()
Y = data[["DEFUNCION"]].to_numpy()


In [4]:
n_experiments = 10
experiments = []

for _ in range(n_experiments):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)

    experiment = f"experiment_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    experiments.append(experiment)

    clf = ak.StructuredDataClassifier(
        overwrite=True, max_trials=5, num_classes=2,
        directory="../autokeras_search", project_name=experiment
    )

    train_history = clf.fit(
        x=X_train, y=Y_train,
        epochs=10, validation_split=0.25
    )

    print(f"========== {experiment} evaluation ==========")
    clf.evaluate(X_test, Y_test)

    model = clf.export_model()
    model.save(f"../models/{experiment}", save_format="tf")

    model.summary()

    del X_train; del X_test; del Y_train; del Y_test
    del clf; del train_history; del model


Trial 5 Complete [00h 03m 12s]
val_accuracy: 0.9798607230186462

Best val_accuracy So Far: 0.9798607230186462
Total elapsed time: 00h 15m 50s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
2337/2337 [==============================] - 17s 7ms/step - loss: 0.1554 - accuracy: 0.9558
Epoch 2/10
2337/2337 [==============================] - 16s 7ms/step - loss: 0.0999 - accuracy: 0.9785
Epoch 3/10
2337/2337 [==============================] - 17s 7ms/step - loss: 0.0941 - accuracy: 0.9791
Epoch 4/10
2337/2337 [==============================] - 17s 7ms/step - loss: 0.0902 - accuracy: 0.9790
Epoch 5/10
2337/2337 [==============================] - 17s 7ms/step - loss: 0.0904 - accuracy: 0.9793
Epoch 6/10
2337/2337 [==============================] - 16s 7ms/step - loss: 0.0880 - accuracy: 0.9790
Epoch 7/10
2337/2337 [==============================] - 16s 7ms/step - loss: 0.0889 - accuracy: 0.9789
Epoch 8/10
2337/2337 [==============================] - 17s 7ms/step - loss: 0.0872 - accuracy: 0.9

In [ ]:
for experiment in experiments: